# Scientifics Packages
Gonzalo Rios (grios@dim.uchile.cl)

http://scikits.appspot.com/scikits

https://www.scipy.org/topical-software.html

# Sympy: Symbolic Mathematics in Python
http://www.sympy.org

In [ ]:
import sympy as sy
x, y = sy.symbols('x y')
x, y

In [ ]:
expr = x + 2*y
expr

In [ ]:
expr + 1

In [ ]:
expr + 1 - x

In [ ]:
x*expr

In [ ]:
sy.expand(x*expr)

In [ ]:
sy.factor(sy.expand(x*expr))

In [ ]:
sy.diff(sy.sin(x)*sy.exp(x), x)

In [ ]:
sy.integrate(sy.exp(x)*sy.sin(x) + sy.exp(x)*sy.cos(x), x)

In [ ]:
r = sy.limit(sy.sin(x)/x, x, 0)
r

In [ ]:
sy.solve(x**2 - 2, x)

In [ ]:
sy.Integral(sy.cos(x)**2, (x, 0, sy.pi))

In [ ]:
sy.pprint(sy.Integral(sy.cos(x)**2, (x, 0, sy.pi)))

In [ ]:
print(sy.latex(sy.Integral(sy.cos(x)**2, (x, 0, sy.pi))))

In [ ]:
%%latex
$\int_{0}^{\pi} \cos^{2}{\left (x \right )}\, dx$

In [ ]:
sy.Integral(sy.cos(x)**2, (x, 0, sy.pi)).evalf()

In [ ]:
expr = sy.diff(sy.sin(x)*sy.exp(x), x)
expr

In [ ]:
expr.evalf(subs={x: 3.14})

In [ ]:
f = sy.lambdify(x, expr)
f

In [ ]:
f(3.14)

In [ ]:
fnp = sy.lambdify(x, expr, 'numpy')
fnp

In [ ]:
fnp(3.14)

In [ ]:
%timeit expr.evalf(subs={x: 3.14})

In [ ]:
%timeit f(3.14)

In [ ]:
%timeit fnp(3.14)

In [ ]:
import numpy as np
x_test = np.random.randn(1000)

In [ ]:
%timeit fnp(x_test)

In [ ]:
%timeit np.array([f(i) for i in x_test])

# StatsModels: Statistic in Python
http://www.statsmodels.org/stable/

In [ ]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

sb.set(context='notebook', rc={"figure.figsize": (20, 8)})

nobs = 10000
X = np.random.random((nobs, 2))
X = sm.add_constant(X)
X

In [ ]:
beta = [2.2, .3, .8]
e = np.random.randn(nobs)*0.2

y = np.dot(X, beta) + e
plt.plot(y, 'ro')
plt.show()
plt.plot(X[:, 0], 'r', label="0")
plt.plot(X[:, 1], 'bo', label="1")
plt.plot(X[:, 2], 'g*', label="2")
plt.legend()

In [ ]:
results = sm.OLS(y, X).fit()
results.summary()

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

try:
    dat = sm.datasets.get_rdataset("data/Guerry", "data/HistData").data
    dat.to_hdf('data/Guerry.h5', key='HistData', mode='w')
    print('Load dataset from R')
except:
    dat = pd.read_hdf('data/Guerry.h5')
    print('Load dataset from HDF file')
dat

In [ ]:
dat.Lottery.plot(style='ro')

In [ ]:
results = smf.ols('Lottery ~ Literacy + Infanticide**2 + np.log(Pop1831)', data=dat).fit()
results.summary()

In [ ]:
results

In [ ]:
print('R2: ', results.rsquared)
print('Parameters: ', results.params)

## Statistics Tests

In [ ]:
results.resid.hist(bins=100)

In [ ]:
from statsmodels.compat import lzip
import statsmodels.stats.api as sms

name = ['Jarque-Bera', 'Chi^2 two-tail prob.', 'Skew', 'Kurtosis']
test = sms.jarque_bera(results.resid)
lzip(name, test)

In [ ]:
name = ['Chi^2', 'Two-tail probability']
test = sms.omni_normtest(results.resid)
lzip(name, test)

In [ ]:
name = ['F statistic', 'p-value']
test = sms.het_goldfeldquandt(results.resid, results.model.exog)
lzip(name, test)

In [ ]:
name = ['t value', 'p value']
test = sms.linear_harvey_collier(results)
lzip(name, test)

In [ ]:
np.linalg.cond(results.model.exog)

In [ ]:
from statsmodels.graphics.regressionplots import plot_leverage_resid2

_ = plot_leverage_resid2(results)

## Non-Linear Regression

In [ ]:
nsample = 50
sig = 0.5
x = np.linspace(0, 20, nsample)
X = np.column_stack((x, np.sin(x), (x-5)**2, np.ones(nsample)))
beta = [0.5, 0.5, -0.02, 5.]

y_true = np.dot(X, beta)
y = y_true + sig * np.random.normal(size=nsample)
plt.plot(y, 'ro')

In [ ]:
res = sm.OLS(y, X).fit()
res.summary()

In [ ]:
print('Parameters: ', res.params)
print('Standard errors: ', res.bse)
print('Predicted values: ', res.predict())

In [ ]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std

prstd, iv_l, iv_u = wls_prediction_std(res)

plt.plot(y, 'o', label="data")
plt.plot(y_true, 'b-', label="True")
plt.plot(res.fittedvalues, 'r--.', label="OLS")
plt.plot(iv_u, 'r--')
plt.plot(iv_l, 'r--')
plt.legend(loc='best')

## Heteroscedasticity

In [ ]:
nsample = 50
x = np.linspace(0, 20, nsample)
X = np.column_stack((x, (x - 5)**2))
X = sm.add_constant(X)
beta = [5., 0.5, -0.01]
sig = 0.5
w = np.ones(nsample)
w[nsample * 6//10:] = 3
y_true = np.dot(X, beta)
e = np.random.normal(size=nsample)
y = y_true + sig * w * e 
X = X[:,[0,1]]

In [ ]:
plt.plot(y, '*')

In [ ]:
mod_wls = sm.WLS(y, X, weights=1./w)
res_wls = mod_wls.fit()
res_wls.summary()

In [ ]:
res_ols = sm.OLS(y, X).fit()
res_ols.summary()

In [ ]:
print('OLS', res_ols.params)
print('WLS', res_wls.params)

In [ ]:
prstd_ols, iv_l_ols, iv_u_ols = wls_prediction_std(res_ols)
prstd, iv_l, iv_u = wls_prediction_std(res_wls)

fig, ax = plt.subplots(figsize=(8,6))
ax.plot(x, y, 'o', label="Data")
ax.plot(x, y_true, 'b-', label="True")
# OLS
ax.plot(x, res_ols.fittedvalues, 'r--')
ax.plot(x, iv_u_ols, 'r--', label="OLS")
ax.plot(x, iv_l_ols, 'r--')
# WLS
ax.plot(x, res_wls.fittedvalues, 'g--.')
ax.plot(x, iv_u, 'g--', label="WLS")
ax.plot(x, iv_l, 'g--')
ax.legend(loc="best");

## Autoregressive Moving Average (ARMA)
http://www.statsmodels.org/dev/examples/notebooks/generated/tsa_arma_0.html

In [ ]:
dta = sm.datasets.sunspots.load_pandas().data
dta.index = pd.Index(sm.tsa.datetools.dates_from_range('1700', '2008'))
del dta["YEAR"]
dta.plot()

In [ ]:
_ = sm.graphics.tsa.plot_acf(dta.values.squeeze(), lags=40)
_ = sm.graphics.tsa.plot_pacf(dta, lags=40)

In [ ]:
import statsmodels.tsa.arima.model as model

In [ ]:
arma_mod20 = sm.tsa.ARMA(dta, (2,0)).fit(disp=False)
print(arma_mod20.params)

In [ ]:
arma_mod30 = sm.tsa.ARMA(dta, (3,0)).fit(disp=False)
print(arma_mod30.params)

In [ ]:
print(arma_mod20.aic, arma_mod20.bic, arma_mod20.hqic)

In [ ]:
print(arma_mod30.aic, arma_mod30.bic, arma_mod30.hqic)

In [ ]:
sm.stats.durbin_watson(arma_mod30.resid.values)

In [ ]:
arma_mod30.model.endog_names

In [ ]:
arma_mod30.model.exog_names

In [ ]:
_ = arma_mod20.plot_predict()

In [ ]:
_ = arma_mod30.plot_predict()

In [ ]:
arma_mod20.resid.plot()

In [ ]:
arma_mod30.resid.plot()

In [ ]:
plt.plot(arma_mod30.forecast(100)[0])
plt.plot(arma_mod30.forecast(100)[2])

## Seasonal Decompose

In [ ]:
co2 = pd.DataFrame(sm.datasets.co2.data.load().data)

co2.index = pd.to_datetime(co2['index'])
del co2['index']
co2.plot()

In [ ]:
co2.fillna(method='ffill', inplace=True)
co2.plot()

In [ ]:
decomp = sm.tsa.seasonal_decompose(co2, model='additive')
additive = pd.DataFrame(index=co2.index)
additive['Real'] = decomp.observed
additive['Trend'] = decomp.trend
additive['Seasonal'] = decomp.seasonal
additive['Resid'] = decomp.resid

_ = additive.plot(subplots=True, layout=(2, 2), figsize=(20, 10))

In [ ]:
decomp = sm.tsa.seasonal_decompose(co2, model='multiplicative')
multiplicative = pd.DataFrame(index=co2.index)
multiplicative['Real'] = decomp.observed
multiplicative['Trend'] = decomp.trend
multiplicative['Seasonal'] = decomp.seasonal
multiplicative['Resid'] = decomp.resid

_ = multiplicative.plot(subplots=True, layout=(2, 2), figsize=(20, 10))

In [ ]:
models = pd.DataFrame(index=co2.index)
models['Real'] = decomp.observed
models['Additive'] = additive['Trend'] + additive['Seasonal']
models['Multiplicative'] = multiplicative['Trend']*multiplicative['Seasonal']
models.plot()

In [ ]:
np.mean((models['Real'] - models['Additive'])**2)

In [ ]:
np.mean((models['Real'] - models['Multiplicative'])**2)

# Scikit Learn: Machine Learning in Python
## Clustering

In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk

from sklearn import cluster, mixture
n_samples = 500

C = np.array([[0., -0.1, 3.0], [1.7, .4, 3.1]])
X = np.concatenate([np.dot(np.random.randn(n_samples, 2), C),
                   .7 * np.random.randn(n_samples, 3) + np.array([-6, 3, 2])])
X = pd.DataFrame(X, columns=['x1', 'x2', 'x3'])
X

In [ ]:
kmeans = cluster.KMeans(n_clusters=2).fit(X)
kmeans

In [ ]:
kmeans.predict(X)

In [ ]:
kmeans.cluster_centers_

In [ ]:
%matplotlib inline
import itertools
import matplotlib.pyplot as plt


def plot_clusters(X, cluster, title):
    for ids in [[0, 1], [0, 2], [1, 2]]:
        plt.figure(figsize=(15, 3))
        plot = plt.subplot(111)
        color_iter = itertools.cycle(['navy', 'c', 'cornflowerblue', 'gold', 'darkorange', 'r', 'g', 'k', 'm'])
        for i, color in zip(range(min(cluster), max(cluster)+1), color_iter):
            if not np.any(cluster == i):
                continue
            plt.scatter(X[cluster == i, ids[0]], X[cluster == i, ids[1]], .8, color=color)
        plt.title(title + str(ids))

In [ ]:
plot_clusters(X.values, kmeans.predict(X), 'KMeans')

In [ ]:
kmeans = cluster.KMeans(n_clusters=10).fit(X)

plot_clusters(X.values, kmeans.predict(X), 'KMeans')

In [ ]:
agglomerative = cluster.AgglomerativeClustering(n_clusters=10, linkage='ward').fit(X)

plot_clusters(X.values, agglomerative.fit_predict(X), 'AgglomerativeClustering')

In [ ]:
birch = cluster.Birch(branching_factor=50, n_clusters=10, threshold=0.5, compute_labels=True).fit(X)
plot_clusters(X.values, birch.predict(X), 'Birch')

In [ ]:
affinity = cluster.AffinityPropagation(preference=-100).fit(X)

plot_clusters(X.values, affinity.predict(X), 'AffinityPropagation')

In [ ]:
dbscan = cluster.DBSCAN(eps=0.5, min_samples=10).fit(X)
plot_clusters(X.values, dbscan.fit_predict(X), 'DBSCAN')

In [ ]:
bandwidth = cluster.estimate_bandwidth(X.values, quantile=0.01, n_samples=500)
meanshift = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True).fit(X)

plot_clusters(X.values, meanshift.predict(X), 'MeanShift')

In [ ]:
gmm = mixture.GaussianMixture(n_components=10, covariance_type='full').fit(X)
plot_clusters(X.values, gmm.predict(X), 'Gaussian Mixture')

In [ ]:
dpgmm = mixture.BayesianGaussianMixture(n_components=10, covariance_type='full').fit(X)
plot_clusters(X.values, dpgmm.predict(X), 'Bayesian Gaussian Mixture with a Dirichlet process prior')

## Classification

In [ ]:
np.c_?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import datasets
from sklearn.neighbors import NearestCentroid

n_neighbors = 15

# import some data to play with
iris = datasets.load_iris()
# we only take the first two features. We could avoid this ugly
# slicing by using a two-dim dataset
X = iris.data[:, :2]
y = iris.target

h = .02  # step size in the mesh

# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

for shrinkage in [None, 1.0, .2, 0.1]:
    # we create an instance of Neighbours Classifier and fit the data.
    clf = NearestCentroid(shrink_threshold=shrinkage)
    clf.fit(X, y)
    y_pred = clf.predict(X)

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(10, 5))
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light, shading='auto')

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold,
                edgecolor='b', s=20)
    plt.title("3-Class classification (shrink_threshold={}, precision={})".format(shrinkage, np.mean(y == y_pred)))
    plt.axis('tight')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets

n_neighbors = 15

# import some data to play with
iris = datasets.load_iris()

# we only take the first two features. We could avoid this ugly
# slicing by using a two-dim dataset
X = iris.data[:, :2]
y = iris.target

h = .02  # step size in the mesh

# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

for weights in ['uniform', 'distance']:
    # we create an instance of Neighbours Classifier and fit the data.
    clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
    clf.fit(X, y)

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(10, 5))
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light, shading='auto')

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold,
                edgecolor='k', s=20)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title("3-Class classification (k = %i, weights = '%s')" % (n_neighbors, weights))

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from sklearn import clone
from sklearn.datasets import load_iris
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                              AdaBoostClassifier)
from sklearn.tree import DecisionTreeClassifier

# Parameters
n_classes = 3
n_estimators = 30
cmap = plt.cm.RdYlBu
plot_step = 0.02  # fine step width for decision surface contours
plot_step_coarser = 0.5  # step widths for coarse classifier guesses
RANDOM_SEED = 13  # fix the seed on each iteration

# Load data
iris = load_iris()

plot_idx = 1

models = [DecisionTreeClassifier(max_depth=None),
          RandomForestClassifier(n_estimators=n_estimators),
          ExtraTreesClassifier(n_estimators=n_estimators),
          AdaBoostClassifier(DecisionTreeClassifier(max_depth=3),
                             n_estimators=n_estimators)]
plt.figure(figsize=(20, 15))
for pair in ([0, 1], [0, 2], [2, 3]):
    for model in models:
        # We only take the two corresponding features
        X = iris.data[:, pair]
        y = iris.target

        # Shuffle
        idx = np.arange(X.shape[0])
        np.random.seed(RANDOM_SEED)
        np.random.shuffle(idx)
        X = X[idx]
        y = y[idx]

        # Standardize
        mean = X.mean(axis=0)
        std = X.std(axis=0)
        X = (X - mean) / std

        # Train
        clf = clone(model)
        clf = model.fit(X, y)

        scores = clf.score(X, y)
        # Create a title for each column and the console by using str() and
        # slicing away useless parts of the string
        model_title = str(type(model)).split(
            ".")[-1][:-2][:-len("Classifier")]

        model_details = model_title
        if hasattr(model, "estimators_"):
            model_details += " with {} estimators".format(
                len(model.estimators_))
        print(model_details + " with features", pair,
              "has a score of", scores)

        plt.subplot(3, 4, plot_idx)
        if plot_idx <= len(models):
            # Add a title at the top of each column
            plt.title(model_title)

        # Now plot the decision boundary using a fine mesh as input to a
        # filled contour plot
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                             np.arange(y_min, y_max, plot_step))

        # Plot either a single DecisionTreeClassifier or alpha blend the
        # decision surfaces of the ensemble of classifiers
        if isinstance(model, DecisionTreeClassifier):
            Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
            Z = Z.reshape(xx.shape)
            cs = plt.contourf(xx, yy, Z, cmap=cmap)
        else:
            # Choose alpha blend level with respect to the number
            # of estimators
            # that are in use (noting that AdaBoost can use fewer estimators
            # than its maximum if it achieves a good enough fit early on)
            estimator_alpha = 1.0 / len(model.estimators_)
            for tree in model.estimators_:
                Z = tree.predict(np.c_[xx.ravel(), yy.ravel()])
                Z = Z.reshape(xx.shape)
                cs = plt.contourf(xx, yy, Z, alpha=estimator_alpha, cmap=cmap)

        # Build a coarser grid to plot a set of ensemble classifications
        # to show how these are different to what we see in the decision
        # surfaces. These points are regularly space and do not have a
        # black outline
        xx_coarser, yy_coarser = np.meshgrid(
            np.arange(x_min, x_max, plot_step_coarser),
            np.arange(y_min, y_max, plot_step_coarser))
        Z_points_coarser = model.predict(np.c_[xx_coarser.ravel(),
                                         yy_coarser.ravel()]
                                         ).reshape(xx_coarser.shape)
        cs_points = plt.scatter(xx_coarser, yy_coarser, s=15,
                                c=Z_points_coarser, cmap=cmap,
                                edgecolors="none")

        # Plot the training points, these are clustered together and have a
        # black outline
        plt.scatter(X[:, 0], X[:, 1], c=y,
                    cmap=ListedColormap(['r', 'y', 'b']),
                    edgecolor='k', s=20)
        plot_idx += 1  # move on to the next plot in sequence

plt.suptitle("Classifiers on feature subsets of the Iris dataset")
plt.axis("tight")

plt.show()

## Dimensionality Reduction

In [ ]:
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

iris = datasets.load_iris()

X = iris.data
y = iris.target
target_names = iris.target_names

pca = PCA(n_components=2)
X_r = pca.fit(X).transform(X)

lda = LinearDiscriminantAnalysis(n_components=2)
X_r2 = lda.fit(X, y).transform(X)

# Percentage of variance explained for each components
print('explained variance ratio (first two components): %s' % str(pca.explained_variance_ratio_))

plt.figure()
colors = ['navy', 'turquoise', 'darkorange']
lw = 2

for color, i, target_name in zip(colors, [0, 1, 2], target_names):
    plt.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color, alpha=.8, lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('PCA of IRIS dataset')

plt.figure()
for color, i, target_name in zip(colors, [0, 1, 2], target_names):
    plt.scatter(X_r2[y == i, 0], X_r2[y == i, 1], alpha=.8, color=color,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('LDA of IRIS dataset')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA, KernelPCA
from sklearn.datasets import make_circles

np.random.seed(0)

X, y = make_circles(n_samples=400, factor=.3, noise=.05)

plt.figure(figsize=(12, 12))
plt.subplot(2, 2, 1, aspect='equal')
plt.title("Original space")
reds = y == 0
blues = y == 1
plt.scatter(X[reds, 0], X[reds, 1], c="red", s=20, edgecolor='k')
plt.scatter(X[blues, 0], X[blues, 1], c="blue", s=20, edgecolor='k')
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")

In [ ]:
kpca = KernelPCA(kernel="rbf", fit_inverse_transform=True, gamma=10)
X_kpca = kpca.fit_transform(X)
X_back = kpca.inverse_transform(X_kpca)
pca = PCA()
X_pca = pca.fit_transform(X)

# Plot results

plt.figure(figsize=(12, 12))
plt.subplot(2, 2, 1, aspect='equal')
plt.title("Original space")
reds = y == 0
blues = y == 1

plt.scatter(X[reds, 0], X[reds, 1], c="red", s=20, edgecolor='k')
plt.scatter(X[blues, 0], X[blues, 1], c="blue", s=20, edgecolor='k')
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")

X1, X2 = np.meshgrid(np.linspace(-1.5, 1.5, 50), np.linspace(-1.5, 1.5, 50))
X_grid = np.array([np.ravel(X1), np.ravel(X2)]).T
# projection on the first principal component (in the phi space)
Z_grid = kpca.transform(X_grid)[:, 0].reshape(X1.shape)
plt.contour(X1, X2, Z_grid, colors='grey', linewidths=1, origin='lower')

plt.subplot(2, 2, 2, aspect='equal')
plt.scatter(X_pca[reds, 0], X_pca[reds, 1], c="red", s=20, edgecolor='k')
plt.scatter(X_pca[blues, 0], X_pca[blues, 1], c="blue", s=20, edgecolor='k')
plt.title("Projection by PCA")
plt.xlabel("1st principal component")
plt.ylabel("2nd component")

plt.subplot(2, 2, 3, aspect='equal')
plt.scatter(X_kpca[reds, 0], X_kpca[reds, 1], c="red", s=20, edgecolor='k')
plt.scatter(X_kpca[blues, 0], X_kpca[blues, 1], c="blue", s=20, edgecolor='k')
plt.title("Projection by KPCA")
plt.xlabel("1st principal component in space induced by $\phi$")
plt.ylabel("2nd component")

plt.subplot(2, 2, 4, aspect='equal')
plt.scatter(X_back[reds, 0], X_back[reds, 1], c="red", s=20, edgecolor='k')
plt.scatter(X_back[blues, 0], X_back[blues, 1], c="blue", s=20, edgecolor='k')
plt.title("Original space after inverse transform")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")

plt.subplots_adjust(0.02, 0.10, 0.98, 0.94, 0.04, 0.35)

plt.show()

# PyMC3: Bayesian Statistical Modeling and Probabilistic Machine Learning 

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'
import pymc3 as pm

data = 1.43 + 1.23*np.random.randn(1000)

with pm.Model() as model:
    mu = pm.Normal('mu', mu=0, sd=1)
    sd = pm.Exponential('sd', lam=1)
    obs = pm.Normal('obs', mu=mu, sd=sd, observed=data)

In [ ]:
plt.plot(data)

In [ ]:
model

In [ ]:
mu

In [ ]:
sd

In [ ]:
obs

In [ ]:
model.basic_RVs

In [ ]:
model.free_RVs

In [ ]:
model.observed_RVs

In [ ]:
model.deterministics

In [ ]:
with model:
    p = pm.find_MAP()
print(p)

In [ ]:
model.logp

In [ ]:
p = model.test_point.copy()
p['mu'] = 1
p

In [ ]:
with model:
    p = pm.find_MAP(start=p)
print(p)

In [ ]:
import scipy as sp


with model:
    p = pm.find_MAP(method='Powell')
print(p)

In [ ]:
with model:
    trace = pm.sample(10000)

In [ ]:
trace

In [ ]:
trace.varnames

In [ ]:
trace['mu']

In [ ]:
pm.traceplot(trace)

In [ ]:
_ = pm.traceplot(trace[500:])
pm.summary(trace[500:])

In [ ]:
with model:
    trace_m = pm.sample(10000, step=pm.Metropolis())

In [ ]:
_ = pm.traceplot(trace_m[500:])
pm.summary(trace_m[500:])

In [ ]:
with model:
    trace_s = pm.sample(1000, step=pm.Slice())

In [ ]:
_ = pm.traceplot(trace_s[500:])
pm.summary(trace_s[500:])

In [ ]:
with model:
    post_pred = pm.sampling.sample_posterior_predictive(trace_s[500:], samples=100, size=len(data))

In [ ]:
plt.figure()
ax = sb.histplot(post_pred['obs'].mean(axis=1).mean(axis=1), kde=True, label='Posterior predictive means')
ax.axvline(data.mean(), color='r', ls='--', label='True mean')
ax.legend()